# 4.3.5. Ley de Hooke generalizada para materiales ortótropos
|Quién | Correo | Fecha | Qué hizo |
| ---  | ---    | ---   | ---      |
|Juan Nicolás Ramírez Giraldo | <jnramirezg@unal.edu.co> | Octubre 3, 2022 | Conversión de código de MAXIMA a Python|
|Diego Andrés Alvarez Marín | <daalvarez@unal.edu.co> | Marzo 27, 2023 | Conversión de código de MAXIMA a Python|

$$
\newcommand{\ve}[1]{{\vec{\boldsymbol{#1}}}}
\newcommand{\ma}[1]{{\boldsymbol{#1}}}
\newcommand{\hve}[1]{{\,\hat{\!\boldsymbol{#1}}}}
\newcommand{\tr}{\operatorname{tr}}
$$

In [1]:
from sympy import init_printing, latex, symbols, linear_eq_to_matrix, simplify

# Para imprimir bonito
init_printing()
from IPython.display import Math                 
def imprimir (texto1, variable, texto2=""):
    return Math(texto1 +  rf'{latex(variable)}' + texto2)

In [2]:
sx,   sy,  sz = symbols("     sigma_x      sigma_y      sigma_z")
txy, txz, tyz = symbols("      tau_xy       tau_xz       tau_yz")
ex,   ey,  ez = symbols("varepsilon_x varepsilon_y varepsilon_z")
gxy, gxz, gyz = symbols("    gamma_xy     gamma_xz     gamma_yz")

In [3]:
Ex,     Ey,   Ez = symbols("  E_x   E_y   E_z")
Gxy,   Gxz,  Gyz = symbols(" G_xy  G_xz  G_yz ")
nuxy, nuxz, nuyz = symbols("nu_xy nu_xz nu_yz ")
nuyx, nuzx, nuzy = symbols("nu_yx nu_zx nu_zy ")

A partir de las ecuaciones:

\begin{align*}
\varepsilon_x &= \frac{1}{E}\sigma_x - \frac{\nu_{yx}}{E}\sigma_y - \frac{\nu_{zx}}{E}\sigma_z\\
\varepsilon_y &= -\frac{\nu_{xy}}{E}\sigma_x + \frac{1}{E}\sigma_y - \frac{\nu_{zy}}{E}\sigma_z\\
\varepsilon_y &= -\frac{\nu_{xz}}{E}\sigma_x - \frac{\nu_{yz}}{E}\sigma_y + \frac{1}{E}\sigma_z\\
\gamma_{xy}   &= \frac{1}{G_{yz}}\tau_{xy}\\
\gamma_{xz}   &= \frac{1}{G_{xz}}\tau_{xz}\\
\gamma_{yz}   &= \frac{1}{G_{xy}}\tau_{yz}
\end{align*}

In [4]:
ex =       sx/Ex - nuyx*sy/Ey - nuzx*sz/Ez
ey = -nuxy*sx/Ex +      sy/Ey - nuzy*sz/Ez
ez = -nuxz*sx/Ex - nuyz*sy/Ey +      sz/Ez
gyz = tyz/Gyz
gxz = txz/Gxz
gxy = txy/Gxy

Extraemos la matriz de conformidad $\ma{S}$:

In [5]:
S = linear_eq_to_matrix([ex, ey, ez, gyz, gxz, gxy], 
                        [sx, sy, sz, tyz, txz, txy])[0]
imprimir(r"\ma{S} = ", S)

<IPython.core.display.Math object>

Ahora, calculamos la matriz constitutiva $\ma{D}=\ma{S}^{-1}$:

In [11]:
D = S**-1
imprimir(r"\ma{D} = ", D)

<IPython.core.display.Math object>

reemplazamos el término:
$$\Delta = \displaystyle\frac{1-\nu_{yz}\nu_{zy} - \nu_{xz}\nu_{zx} - \nu_{xy}\nu_{yx} - \nu_{xz}\nu_{yx}\nu_{zy} - \nu_{xy}\nu_{yz}\nu_{zx}}{E_x E_y E_z}$$

La variable ```delta_``` contiene la parte derecha de la iguadad y la variable ```delta``` contiene el símbolo $\Delta$:

In [12]:
delta_ = (1-nuyz*nuzy - nuxz*nuzx - nuxy*nuyx - nuxz*nuyx*nuzy - nuxy*nuyz*nuzx)/(Ex*Ey*Ez)
delta = symbols("Delta") 

Substituimos ```delta_``` por ```delta```:

In [13]:
D = simplify(D*delta_/delta).subs(delta_,delta)
imprimir(r"\ma{D} = ", D)

<IPython.core.display.Math object>

De aquí obtenemos que la ley de Hooke generalizada para materiales ortótropos en forma matricial es:

$$
\underbrace{%
\begin{bmatrix}
\sigma_x\\ \sigma_y\\ \sigma_z\\ \tau_{yz}\\ \tau_{xz}\\ \tau_{xy}\\
\end{bmatrix}
}_{\ve{\sigma}}
=
\underbrace{%
\begin{bmatrix}
\frac{1- \nu_{yz} \nu_{zy}}{ E_{y} E_{z} \Delta} 
& \frac{\nu_{yz} \nu_{zx}+ \nu_{yx}}{ E_{y} E_{z} \Delta} 
& \frac{\nu_{yx} \nu_{zy} + \nu_{zx}}{ E_{y} E_{z} \Delta} & 0 & 0 & 0\\
\frac{\nu_{xz} \nu_{zy}+ \nu_{xy}  }{ E_{x} E_{z} \Delta} 
& \frac{1- \nu_{xz} \nu_{zx}}{ E_{x} E_{z} \Delta} 
& \frac{\nu_{zy}+\nu_{xy} \nu_{zx} }{ E_{x} E_{z} \Delta} & 0 & 0 & 0\\
\frac{\nu_{xy} \nu_{yz} + \nu_{xz}}{ E_{x} E_{y} \Delta} 
& \frac{\nu_{yz}+\nu_{xz} \nu_{yx}}{ E_{x} E_{y} \Delta} 
& \frac{1- \nu_{xy} \nu_{yx}}{ E_{x} E_{y} \Delta} & 0 & 0 & 0\\0 & 0 & 0 & G_{yz} & 0 & 0\\
0 & 0 & 0 & 0 & G_{xz} & 0\\
0 & 0 & 0 & 0 & 0 & G_{xy}
\end{bmatrix}}_{\ma{D}}
\underbrace{%
\begin{bmatrix}
\varepsilon_x\\ \varepsilon_y\\ \varepsilon_z\\ \gamma_{yz}\\ \gamma_{xz}\\ \gamma_{xy}\\
\end{bmatrix}}_{\ve{\varepsilon}}$$

Donde:
$$\Delta = \displaystyle\frac{1-\nu_{yz}\nu_{zy} - \nu_{xz}\nu_{zx} - \nu_{xy}\nu_{yx} - 2\nu_{xy}\nu_{yz}\nu_{zx}}{E_x E_y E_z}$$

**Nota:** $\nu_{xz}\nu_{yx}\nu_{zy} + \nu_{xy}\nu_{yz}\nu_{zx} = 2\nu_{xy}\nu_{yz}\nu_{zx}$, pues $\nu_{xz}\nu_{yx}\nu_{zy} = \nu_{xy}\nu_{yz}\nu_{zx}$, gracias a que:

$$\nu_{xy}=\displaystyle\frac{E_x}{E_y}\nu_{yx}$$
$$\nu_{yz}=\displaystyle\frac{E_y}{E_z}\nu_{zy}$$
$$\nu_{zx}=\displaystyle\frac{E_z}{E_x}\nu_{xz}$$